In [ ]:
import os
import torch
import datasets.paths as d_path
import torchvision

from engine.lesion_detection_detr import train_one_epoch, evaluate
from utils.init import reproducibility, clean_memory_get_device
from config import ConfigArgs
from utils.train import EarlyStopper, TrainingInfo, epoch_end_print, get_dataloaders, get_datasets
from utils.plot import plot_losses
from IPython.display import clear_output
from datasets.reflacx.lesion_detection import REFLACXLesionDetectionDataset
from engine.lesion_detection import check_best, end_train, load_backbone
from models.detr.detr import build_detr

from utils.train import set_weights_trainable
import warnings
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings("ignore")

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
device = clean_memory_get_device()
# device = torch.device("cpu")
reproducibility()
if torch.cuda.is_available():
    print(torch.cuda.memory_summary())

In [ ]:
from config.dataset import REFLACXLesionDetectionDatasetArgs
from config.model import DETRArgs, FasterRCNNArgs
from config.training import LesionDetectionArgs

training_args = LesionDetectionArgs(
    name="lesion_detection",
    batch_size=4,
    early_stopping_patience = 10,
)
dataset_args = REFLACXLesionDetectionDatasetArgs(
    image_size=128,
)

cl_model_name = "MCL_resnet50_accuracy_0_1433_epoch48_10-08-2023 15-58-25"

configs = [
    # ConfigArgs(
    #     training=training_args,
    #     dataset=dataset_args,
    #     model=FasterRCNNArgs(
    #         name="CL_NoFix",
    #         weights="cl",
    #         cl_model_name=cl_model_name,
    #         trainable_backbone_layers=5,
    #     )
    # ),
    # ConfigArgs(
    #     training=training_args,
    #     dataset=dataset_args,
    #     model=FasterRCNNArgs(
    #         name="CL_Fix5",
    #         weights="cl",
    #         cl_model_name=cl_model_name,
    #         trainable_backbone_layers=0,
    #     )
    # ),
    
    ConfigArgs(
        training=training_args,
        dataset=dataset_args,
        model=DETRArgs(
            name="CL_Fix5Layers",
            weights="cl",
            cl_model_name=cl_model_name,
            trainable_backbone_layers=0,
        ),
    ),

    # ConfigArgs(
    #     training=training_args,
    #     dataset=dataset_args,
    #     model=FasterRCNNArgs(
    #         name="ImageNet_Fix5layers",
    #         weights="imagenet",
    #         trainable_backbone_layers=0,
    #     )
    # ),

    # ConfigArgs(
    #     training=training_args,
    #     dataset=dataset_args,
    #     model=FasterRCNNArgs(
    #         name="CL_Fix2Layers",
    #         weights="cl",
    #         cl_model_name=cl_model_name,
    #         trainable_backbone_layers=3,
    #     )
    # ),
    # ConfigArgs(
    #     training=training_args,
    #     dataset=dataset_args,
    #     model=FasterRCNNArgs(
    #         name="ImageNet_Fix2Layers",
    #         weights="imagenet",
    #         trainable_backbone_layers=3,
    #     )
    # ),
    # ConfigArgs(
    #     training=training_args,
    #     dataset=dataset_args,
    #     model=FasterRCNNArgs(
    #         name="random",
    #         weights=None,
    #         trainable_backbone_layers=5,
    #     )
    # ),
]

In [ ]:
# boxes = box_xyxy_to_cxcywh(boxes) / self.image_size

In [ ]:
num_epochs = 200  # 200
train_infos: list[TrainingInfo] = []

In [ ]:
# # load this one into
# backbone = load_backbone(config, device)
# backbone.body
# model.backbone[0].body = backbone.body

In [ ]:
for config in configs:
    device = clean_memory_get_device()
    reproducibility()

    train_info = TrainingInfo(config)

    model, criterion, postprocessors = build_detr(
        backbone="resnet18",
        train_backbone="true",
        hidden_dim=config.model.hidden_dim,
        dilation=config.model.dilation,
        position_embedding=config.model.position_embedding,
        dropout=config.model.dropout,
        nheads=config.model.nheads,
        dim_feedforward=config.model.dim_feedforward,
        enc_layers=config.model.enc_layers,
        dec_layers=config.model.dec_layers,
        pre_norm=config.model.pre_norm,
        num_classes=len(config.dataset.label_cols),
        num_queries=config.model.num_queries,
        aux_loss=config.model.aux_loss,
        set_cost_class=config.model.set_cost_class,
        set_cost_bbox=config.model.set_cost_bbox,
        set_cost_giou=config.model.set_cost_giou,
        giou_loss_coef=config.model.giou_loss_coef,
        bbox_loss_coef=config.model.bbox_loss_coef,
        eos_coef=config.model.eos_coef,
        device=device,
    )

    # weights.
    backbone = load_backbone(config, device)
    model.backbone[0].body = backbone.body
    # model.backbone = backbone
    model.to(device)

    dataset_args = {
        "df_path": os.path.join("spreadsheets", "reflacx.csv"),
        "mimic_eye_path": d_path.MIMIC_EYE_PATH,
        "image_size": config.dataset.image_size,
        "label_cols": config.dataset.label_cols,
        "cxcywh": True,
    }

    train_dataset, val_dataset, _ = get_datasets(
        dataset_args=dataset_args,
        dataset_class=REFLACXLesionDetectionDataset,
    )

    train_dataloader, val_dataloader = get_dataloaders(
        train_dataset,
        val_dataset,
        batch_size=config.training.batch_size,
        drop_last=False,
    )

    optimizer = torch.optim.AdamW(
        params=[p for p in model.parameters() if p.requires_grad],
        lr=config.training.learning_rate,
        weight_decay=config.training.weight_decay,
    )

    early_stopper = None
    if config.training.early_stopping_patience:
        early_stopper = EarlyStopper(patience=config.training.early_stopping_patience)

    train_info.timer.start_training()

    for e in range(1, num_epochs + 1):
        train_info.epoch = e
        train_info.timer.start_epoch()
        for t_i in train_infos:
            print(t_i)
        print(train_info)

        if (not config.model.release_fixed_weights_after is None) and (
            e > config.model.release_fixed_weights_after
        ):
            model, optimizer = set_weights_trainable(
                model.backbone[0].body, optimizer, None
            )

        train_logger = train_one_epoch(
            model=model,
            criterion=criterion,
            optimizer=optimizer,
            data_loader=train_dataloader,
            device=device,
            epoch=train_info.epoch,
        )

        train_info.train_losses.append(train_logger.get_data())
        model.eval()

        val_logger, val_evaluator = evaluate(
            model=model,
            criterion=criterion,
            data_loader=val_dataloader,
            postprocessors=postprocessors,
            device=device,
            return_evaluator=True,
        )

        train_info.val_losses.append(val_logger.get_data())
        clear_output()
        plot_losses(
            train_info.train_losses,
            train_info.val_losses,
        )
        epoch_end_print(train_info, early_stopper, num_epochs)

        if early_stopper and train_info.epoch > config.training.warmup_epoch:
            train_info = check_best(
                train_info=train_info,
                model=model,
                optimiser=optimizer,
                val_evaluator=val_evaluator,
            )
            to_stop = early_stopper.early_stop(train_info.val_losses[-1]["loss"])
            if to_stop:
                print(
                    f"| Patience reached [{early_stopper.counter}], EarlyStopping end |"
                )
                break

    train_info = end_train(
        train_info=train_info,
        model=model,
        optimiser=optimizer,
        val_evaluator=val_evaluator,
    )

    train_infos.append(train_info)

In [ ]:
for t in train_infos :
    print(t)
    plot_losses(
        t.train_losses,
        t.val_losses,
    )

In [ ]:
# Evaluate all train_infos on the best
from utils.checkpoint import load_checkpoints
from engine.lesion_detection import get_ap_ar

performances = {}

for t in train_infos:
    train_info, cp = load_checkpoints(t.best_val_loss_model_path, device)
    config = train_info.config

    model, criterion, postprocessors = build_detr(
        backbone="resnet18",
        train_backbone="true",
        hidden_dim=config.model.hidden_dim,
        dilation=config.model.dilation,
        position_embedding=config.model.position_embedding,
        dropout=config.model.dropout,
        nheads=config.model.nheads,
        dim_feedforward=config.model.dim_feedforward,
        enc_layers=config.model.enc_layers,
        dec_layers=config.model.dec_layers,
        pre_norm=config.model.pre_norm,
        num_classes=len(config.dataset.label_cols),
        num_queries=config.model.num_queries,
        aux_loss=config.model.aux_loss,
        set_cost_class=config.model.set_cost_class,
        set_cost_bbox=config.model.set_cost_bbox,
        set_cost_giou=config.model.set_cost_giou,
        giou_loss_coef=config.model.giou_loss_coef,
        bbox_loss_coef=config.model.bbox_loss_coef,
        eos_coef=config.model.eos_coef,
        device=device,
    )

    # weights.
    backbone = load_backbone(config, device)
    model.backbone[0].body = backbone.body
    model.to(device)
    model.load_state_dict(cp["model"])
    model.eval()

    dataset_args = {
        "df_path": os.path.join("spreadsheets", "reflacx.csv"),
        "mimic_eye_path": d_path.MIMIC_EYE_PATH,
        "image_size": config.dataset.image_size,
        "label_cols": config.dataset.label_cols,
    }

    _, _, test_dataset = get_datasets(
        dataset_args=dataset_args,
        dataset_class=REFLACXLesionDetectionDataset,
    )

    _, _, test_dataloader = get_dataloaders(
        train_dataset,
        val_dataset,
        test_dataset,
        batch_size=config.training.batch_size,
        drop_last=False,
    )

    _, test_evaluator = evaluate(
        criterion=criterion,
        postprocessors=postprocessors,
        model=model,
        data_loader=test_dataloader,
        device=device,
        return_evaluator=True,
    )

    performance_dict = get_ap_ar(
        test_evaluator.coco_eval["bbox"],
    )

    print(t)
    print(performance_dict)

    performances[f"{config.training.name} - {config.model.name}"] = performance_dict

    with open(
        os.path.join("checkpoints", t.best_val_loss_model_path, "performance.txt"), "w"
    ) as f:
        f.write(str(t) + "\n" + "Best: \n" + str(performance_dict))

In [ ]:
p_df = pd.DataFrame(performances).transpose()

In [ ]:
p_df

In [ ]:
param_dict = {}
for n, p in model.named_parameters():
    param_dict.update({n: {"#params":p.nelement()}})

In [ ]:
pd.DataFrame(param_dict).transpose().sort_values("#params", ascending=False).head(20)